# Pru Wealth Data Test

Below we will explore a mock database of customers, policies and funds that are similar to the Pru Wealth data. The first section of this notebook will test your SQL skills, the second section will test your Python skills and visualisation skills.

The database file is called Prudential_Data.db

The three tables included in this database are:
* Customer_Table
* Policy_Table
* Fund_Table

You will need to have the following packages installed in your Python environment:
* numpy
* pandas
* sqlite3

For visualization use any package of your choice (e.g., matplotlib, seaborn, plotly).

Please complete and return the test before the deadline by emailing your copy of this notebook to: Denholm.Hesse@Prudential.co.uk. If you do not know the answer to a question feel free to use Google and Stack Overflow. This is to give you the option to teach yourself a concept from the internet to address a question which you do not know the answer to, please refrain from skipping questions. The test is meant to be completed by you only.

Good luck and have fun!

In [1]:
import numpy as np
import pandas as pd
import sqlite3

# SQL test

IMPORANT: Please make sure all questions in this section are solved using SQL by only editing the content of the variable *sqlite_query*. Please do not edit the below cell:

In [2]:
# Establish DB connection
conn = sqlite3.connect('Prudential_Data.db')

Below is an example of how to query the SQLlite databse from this Jupyter notebook. Please note that SQLlite might have a slightly different syntax from the SQL you normally use. Please do not edit the below cell:

In [3]:
sqlite_query = """
SELECT COUNT(*) 
from policy_table 
"""

pd.read_sql(sqlite_query, conn)

,COUNT(*)
0,1635


The first table "Customer_Table" contains personal information about the customers. Please do not edit the below cell:

In [4]:
sqlite_query = """
SELECT * 
from Customer_Table 
limit 5
"""

pd.read_sql(sqlite_query, conn)

,index,Customer ID,Email,Gender,Age,Postcode
0,0,CO183683,NLUPPITT14@PARALLELS.COM,Male,90,None
1,1,CO314757,None,Female,90,None
2,2,CO720602,None,Female,90,None
3,3,CO34804,None,Female,90,1495-701
4,4,CO474411,None,Male,90,301288


The second table "Policy_Table" contains data relating to customer policies. The policy value is the current value of the policy. Please do not edit the below cell:

In [5]:
sqlite_query = """
SELECT * 
from Policy_Table 
limit 5
"""

pd.read_sql(sqlite_query, conn)

,index,Policy Number,Customer ID,Product Category,Product Name,Policy Value,Policy Start Date
0,0,RET100072,CO276506,RETIREMENT,RETIREMENT ACCOUNT,270033.0,2016-01-02 00:00:00
1,1,RET100931,CO831289,RETIREMENT,RETIREMENT ACCOUNT,26394.0,2016-01-04 00:00:00
2,2,RET100682,CO784249,RETIREMENT,RETIREMENT ACCOUNT,773567.0,2016-01-04 00:00:00
3,3,RET100447,CO295533,RETIREMENT,RETIREMENT ACCOUNT,35512.0,2016-01-07 00:00:00
4,4,RET100390,CO901735,RETIREMENT,RETIREMENT ACCOUNT,767580.0,2016-01-08 00:00:00


The final table "Fund_Table" contains historic data on the Policy Value (Policy_Value_Snapshot). The Policy_Value_Snapshot represents the value of the policy and the Date column is the date at which that value was recorded. The value may have changed overtime due to changes in the fund price or quanity of units the policy was invested in. Please do not edit the below cell:

In [6]:
sqlite_query = """
SELECT 
*
from Fund_Table
limit 5

"""

pd.read_sql(sqlite_query, conn)

,index,Policy Number,Customer ID,Date,Product Category,Product Name,Policy_Value_Snapshot
0,0,ANN1221,CO408564,2009-10-31 00:00:00,ANNUITY,STANDARD ANNUITY,None
1,1,ANN1221,CO408564,2009-11-30 00:00:00,ANNUITY,STANDARD ANNUITY,None
2,2,ANN1221,CO408564,2009-12-31 00:00:00,ANNUITY,STANDARD ANNUITY,None
3,3,ANN1221,CO408564,2010-01-31 00:00:00,ANNUITY,STANDARD ANNUITY,None
4,4,ANN1221,CO408564,2010-02-28 00:00:00,ANNUITY,STANDARD ANNUITY,None


If a question requires additional explanation for your answer, please add additional cells below the code. 

## Q1: What is the current average policy value split by each product name? 

In [36]:
sqlite_query =  """select F."Product Name", AVG(Policy_Value_Snapshot) FROM 
( SELECT "Customer ID", "Product Name", MAX("Date") "Date" 
FROM FUND_TABLE F
GROUP BY 
"Customer ID", "Product Name"
)F,

FUND_TABLE F2

WHERE F."Customer ID"=F2."Customer ID"
AND F."Product Name" = F2."Product Name"
AND F."Date"=F2."Date"

GROUP BY 
F."Product Name";

"""

pd.read_sql(sqlite_query, conn)

,Product Name,AVG(Policy_Value_Snapshot)
0,CORPORATE MEMBERSHIP,5697.285714
1,FOCUSSED ISA,5746.341270
2,NHS PENSION,5241.500000
3,PRU BOND,4516.389831
4,PRUDENTIAL INVESTMENT PLAN,5084.515152
5,PRUDENTIAL ISA,5161.135593
6,RETIREMENT ACCOUNT,504964.959960
7,SIPP,5335.842466
8,STANDARD ANNUITY,NaN


## Q2: How many customers opened a new policy in Q1 2017 who already had an existing pension product? 

In [79]:
sqlite_query = """

Select Count (Distinct "Customer ID")
from 
Policy_Table

where strftime('%Y',"Policy Start Date") = "2017"
and strftime ('%m' , "Policy Start Date") IN ('01','02','03')
and "Customer ID" IN 

(
Select "Customer ID" from Policy_Table 
where "Product Category" = "PENSION"
and strftime ('%Y' , "Policy Start Date")<='2016'
)


"""
pd.read_sql(sqlite_query, conn)

,"Count (Distinct ""Customer ID"")"
0,35


## Q3: What was the customer ID that had the biggest difference in policy value between its earliest and latest date (for any product) ?

In [264]:
sqlite_query = """


With CTE1 AS
(
select "Customer ID", "Policy Number", "Policy_Value_Snapshot"
from
(
select "Customer ID", "Policy Number", "Policy_Value_Snapshot", "Date", rank() over(partition by "Customer ID", "Policy Number" order by "Date") as rnk
from Fund_Table
) T1
where rnk=1
)
,
CTE2 AS
(
select "Customer ID", "Policy Number", "Policy_Value_Snapshot"
from
(
select "Customer ID", "Policy Number", "Policy_Value_Snapshot", "Date", rank() over(partition by "Customer ID", "Policy Number" order by "Date" DESC) as rnk
from Fund_Table
) T2
where rnk=1
)


select distinct "Customer ID"
from
(
Select CTE1."Customer ID", rank() over( order by CTE2."Policy_Value_Snapshot"-CTE1."Policy_Value_Snapshot" DESC) as rnk1
from CTE1 Inner join CTE2
On CTE1."Customer ID"=CTE2."Customer ID" and CTE1."Policy Number"=CTE2."Policy Number"
) T
where T.rnk1=1

"""

pd.read_sql(sqlite_query, conn)

,Customer ID
0,CO401070


# Python Test

Same data, but now let's have some fun in python. Please do not use SQL for this part of the exercise. 

Feel free to add more cells of code or text

In [84]:
Customer_Table = pd.read_sql('SELECT * FROM Customer_Table', conn)
Policy_Table = pd.read_sql('SELECT * FROM Policy_Table', conn)
Fund_Table = pd.read_sql('SELECT * FROM Fund_Table', conn)

## Q4: Produce a list of customers which have an invalid email address. What do you notice about these customers? ##

Feel free to use any packages of your choice

In [124]:
#Insert your answer below: 


In [358]:
#Dropping the Null values from the Email Column as we are focusing on the invalid email addresses and not missing
new_Customer_Table=Customer_Table.dropna(subset=['Email']).reset_index()
new_Customer_Table=new_Customer_Table.drop('level_0',axis=1)

In [359]:
new_Customer_Table['Email'].value_counts()

NOEMAIL@MAIL.COM                3
EDAWLTREY8Q@NETSCAPE.COM        1
MDEDAM7J@CMU.EDU                1
SYUSHMANOVOG@ABOUT.COM          1
DLUCIENFO@SHUTTERFLY.COM        1
                               ..
SHODGENY@1688.COM               1
JDELATOURQ4@BUSINESSWIRE.COM    1
GHEYMESD8@NEWSVINE.COM          1
FLINDSELLIW@BOSTON.COM          1
SGRACEY8F@MOZILLA.ORG           1
Name: Email, Length: 950, dtype: int64

In [360]:
#There are 3 customers with NOEMAIL@MAIL.COM, which needs further attendtion,
#this can be auto generated inplace of missing data, but I am assuming it as invalid.

df1=new_Customer_Table[new_Customer_Table['Email'].str.match('NOEMAIL@MAIL.COM')]

In [361]:
#defining regex to check valid email address.

regex = '^(\w|\.|\_|\-)+[@](\w|\_|\-|\.)+[.]\w{2,3}$'

In [362]:
#Here is the list of customers who has invalid email as per the regex
df2=new_Customer_Table[new_Customer_Table['Email'].str.match(regex)==False]

In [363]:
#concatinating both dataframes to get final dataframe of the customers with invlaid email address
incorrect_email_customer=pd.concat([df1,df2])

Here is the list of customers which have an invalid email address

In [364]:
incorrect_email_customer['Customer ID']

406    CO426799
529    CO634097
902    CO550430
84     CO609712
178     CO71578
179    CO545536
295    CO795420
355    CO205159
883     CO37714
926    CO922481
Name: Customer ID, dtype: object

In [365]:
incorrect_email_customer

,index,Customer ID,Email,Gender,Age,Postcode
406,450,CO426799,NOEMAIL@MAIL.COM,Female,65,62972 CEDEX 9
529,576,CO634097,NOEMAIL@MAIL.COM,Male,62,80130
902,949,CO550430,NOEMAIL@MAIL.COM,Male,41,None
84,121,CO609712,NOEMAIL,Male,81,None
178,218,CO71578,TGADDIEHB@TOPSY,Female,74,None
179,219,CO545536,VPEGGSK3@@@SPRINGER.COM,Male,74,46100-000
295,337,CO795420,.,Female,68,680011
355,399,CO205159,ALOWSELYHF@#AMAZON.CO.JP,Female,65,None
883,930,CO37714,GSTEERSMR@ABOUTADS.INFO,Male,42,2600
926,973,CO922481,CUSTOMER TO FOLLOW UP,Male,33,8340


In [366]:
len(incorrect_email_customer)

10

In [371]:
incorrect_email_customer.merge(Fund_Table, on='Customer ID', how='inner')['Product Name'].value_counts()

RETIREMENT ACCOUNT            191
FOCUSSED ISA                  168
SIPP                          167
STANDARD ANNUITY               69
PRUDENTIAL INVESTMENT PLAN     58
Name: Product Name, dtype: int64

In [372]:
Fund_Table['Product Name'].value_counts()

RETIREMENT ACCOUNT            19321
SIPP                          12115
FOCUSSED ISA                  10564
STANDARD ANNUITY              10155
PRUDENTIAL INVESTMENT PLAN     8156
PRUDENTIAL ISA                 5031
PRU BOND                       4814
CORPORATE MEMBERSHIP           1629
NHS PENSION                     336
Name: Product Name, dtype: int64

The data for these 10 records is distributed like its population.

## Q5: The Managing Director of Pru Wealth needs to see how policy value differs between other variables in the database (e.g. customer demographics) . They don't have time to read a report, so have asked for a cool visualisation that tells a story.

Visualise something cool, tell us a story! You can use packages like Matplotlib, Seaborn or Plotly.Tell us why you chose to visualise the data in this way and how it can be used from a business perspective.


In [132]:
# Insert your answer below: 
import matplotlib.pyplot as plt
import seaborn as sns
import plotly
import cufflinks as cf
import plotly.express as px
import chart_studio.plotly as py
from plotly.offline import download_plotlyjs,init_notebook_mode, plot, iplot
import plotly.graph_objects as go
init_notebook_mode(connected=True)
cf.go_offline()

In [90]:
Customer_Table = pd.read_sql('SELECT * FROM Customer_Table', conn)
Policy_Table = pd.read_sql('SELECT * FROM Policy_Table', conn)
Fund_Table = pd.read_sql('SELECT * FROM Fund_Table', conn)

In [263]:
dfNew = Customer_Table.merge(Policy_Table, on='Customer ID',how='inner', suffixes=('', '_y'))
dfNew.drop(dfNew.filter(regex='_y$').columns.tolist(),axis=1, inplace=True)

In [262]:
Joined_Table = dfNew.merge(Fund_Table, on='Customer ID',how='inner', suffixes=('', '_y'))
Joined_Table.drop(Joined_Table.filter(regex='_y$').columns.tolist(),axis=1, inplace=True)

# Time series visuals

In [335]:
t=Policy_Table.groupby('Policy Start Date')['Policy Number'].count()
t=t.reset_index()
t

,Policy Start Date,Policy Number
0,2009-07-08 00:00:00,1
1,2009-07-10 00:00:00,1
2,2009-07-11 00:00:00,1
3,2009-07-14 00:00:00,1
4,2009-07-19 00:00:00,1
...,...,...
1201,2018-12-27 00:00:00,4
1202,2018-12-28 00:00:00,2
1203,2018-12-29 00:00:00,1
1204,2018-12-30 00:00:00,1


In [336]:
t1=Policy_Table.groupby('Policy Start Date')['Policy Value'].sum()
t1=t1.reset_index()
t1

,Policy Start Date,Policy Value
0,2009-07-08 00:00:00,2423.0
1,2009-07-10 00:00:00,6384.0
2,2009-07-11 00:00:00,6981.0
3,2009-07-14 00:00:00,5058.0
4,2009-07-19 00:00:00,2179.0
...,...,...
1201,2018-12-27 00:00:00,2477571.0
1202,2018-12-28 00:00:00,1359095.0
1203,2018-12-29 00:00:00,909480.0
1204,2018-12-30 00:00:00,891133.0


In [348]:
time=pd.merge(t, t1).reset_index()
time['year'] = pd.DatetimeIndex(time['Policy Start Date']).year
time['month'] = pd.DatetimeIndex(time['Policy Start Date']).month
time['quarter'] = pd.DatetimeIndex(time['Policy Start Date']).quarter

time

,index,Policy Start Date,Policy Number,Policy Value,year,month,quarter
0,0,2009-07-08 00:00:00,1,2423.0,2009,7,3
1,1,2009-07-10 00:00:00,1,6384.0,2009,7,3
2,2,2009-07-11 00:00:00,1,6981.0,2009,7,3
3,3,2009-07-14 00:00:00,1,5058.0,2009,7,3
4,4,2009-07-19 00:00:00,1,2179.0,2009,7,3
...,...,...,...,...,...,...,...
1201,1201,2018-12-27 00:00:00,4,2477571.0,2018,12,4
1202,1202,2018-12-28 00:00:00,2,1359095.0,2018,12,4
1203,1203,2018-12-29 00:00:00,1,909480.0,2018,12,4
1204,1204,2018-12-30 00:00:00,1,891133.0,2018,12,4


In [340]:
time1=time.groupby('year')['Policy Value', 'Policy Number'].sum()
time1=time1.reset_index()
time1

C:\Users\Asus\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: FutureWarning:

Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.



,year,Policy Value,Policy Number
0,2009,284010.0,65
1,2010,475086.0,119
2,2011,455122.0,109
3,2012,541051.0,113
4,2013,532566.0,128
5,2014,430187.0,102
6,2016,157065199.0,321
7,2017,176332092.0,344
8,2018,170407125.0,332
9,2019,655579.0,2


In [344]:
from plotly.subplots import make_subplots

# Create figure with secondary y-axis
fig = make_subplots(specs=[[{"secondary_y": True}]])

# Add traces
fig.add_trace(
    go.Scatter(x=time1['year'], y=time1['Policy Value'], name="POLICY VALUE"),
    secondary_y=False,
)

fig.add_trace(
    go.Scatter(x=time1['year'], y=time1['Policy Number'], name="NUMBER OF POLICY"),
    secondary_y=True,
)

# Add figure title
fig.update_layout(
    title_text="Current value of the policies and Number of Policies sold per year"
)

# Set x-axis title
fig.update_xaxes(title_text="YEAR")

# Set y-axes titles
fig.update_yaxes(title_text="<b>PRESENT POLICY VALUE", secondary_y=False)
fig.update_yaxes(title_text="<b>NUMBER OF POLICY SOLD", secondary_y=True)

fig.show()


In [349]:
time1=time.groupby('quarter')['Policy Value', 'Policy Number'].sum()
time1=time1.reset_index()
time1

C:\Users\Asus\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: FutureWarning:

Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.



,quarter,Policy Value,Policy Number
0,1,126166846.0,398
1,2,119867880.0,372
2,3,126482101.0,424
3,4,134661190.0,441


In [355]:
fig = px.line(time1, x="quarter", y="Policy Value", title='Policy Value and Quarters')
fig.show()


In [356]:
fig = px.line(time1, x="quarter", y="Policy Number", title='New Policies by Quarter')
fig.show()


# Visuals to understand Policy Value on customers demogrphics

In [216]:
fig = px.histogram(Customer_Table, x='Gender', title="Count of Customers by Gender", color = 'Gender')
fig.show()

In [217]:
fig = px.histogram(dfNew, x='Gender', title="Count of Policies by Gender" ,hover_data=['Policy Value'], color= 'Gender')
fig.show()

Total policy value for the Male policy holders is more than the value for teh female policy holders

In [218]:
fig = px.histogram(dfNew, x='Gender', y='Policy Value' ,title="Count of Policies by Gender" ,color = 'Gender')
fig.show()

The age of the customers is nearly normally distributted, more that 50% of our customers are nearing retirement age.

Our Sales team must 
        1. find a way to retain these customers and offer them relatable products
        2. They should targets the young customers more aggresively to maintain the balance

In [194]:
px.histogram(Customer_Table['Age'],nbins=10, title='Distribution of Age')

In [200]:
px.histogram(Policy_Table['Product Category'], title='Distribution of Product Category among Customers', 
             color=Policy_Table['Product Category'])

999 count of Retirement Policy category states that each customer had opted this policy, which is a good condition for US.

In [211]:
cat=pd.DataFrame(dfNew.groupby(['Product Category','Product Name'])['Policy Value'].sum())
cat=cat.reset_index()

In [215]:
print(cat)

  Product Category                Product Name  Policy Value
0          ANNUITY            STANDARD ANNUITY           0.0
1             BOND                    PRU BOND      266467.0
2             BOND  PRUDENTIAL INVESTMENT PLAN      503367.0
3              ISA                FOCUSSED ISA      724039.0
4              ISA              PRUDENTIAL ISA      304507.0
5          PENSION        CORPORATE MEMBERSHIP      119643.0
6          PENSION                 NHS PENSION       20966.0
7          PENSION                        SIPP      779033.0
8       RETIREMENT          RETIREMENT ACCOUNT   504459995.0


In [212]:
fig = px.bar(cat, x='Product Category',y='Policy Value', title='Policy Value by Product Category', 
             color=cat['Policy Value'], text = 'Policy Value')
fig.update_traces(texttemplate='%{text:.2s}', textposition='outside')
fig.update_layout(uniformtext_minsize=8, uniformtext_mode='hide')
fig.show()

In [213]:
px.histogram(Policy_Table['Product Name'], title='Distribution of Product Name among Customers', color=Policy_Table['Product Name'])

In [214]:
fig = px.bar(cat, x='Product Name',y='Policy Value', title='Policy Value by Product Name', 
             color=cat['Policy Value'], text = 'Policy Value')
fig.update_traces(texttemplate='%{text:.2s}', textposition='outside')
fig.update_layout(uniformtext_minsize=8, uniformtext_mode='hide')
fig.show()

In [160]:
dfNew.groupby("Gender")["Policy Number"].count()

Gender
Female    808
Male      827
Name: Policy Number, dtype: int64

In [192]:
cat=pd.DataFrame(dfNew.groupby('Product Category')['Policy Value'].sum())
cat=cat.reset_index()
cat

,Product Category,Policy Value
0,ANNUITY,0.0
1,BOND,769834.0
2,ISA,1028546.0
3,PENSION,919642.0
4,RETIREMENT,504459995.0


# Working around to see the relation of policy value geographically using available Postcodes

In [289]:
import pgeocode

In [275]:
loc = dfNew.groupby('Postcode')['Age','Policy Value'].median()
loc=loc.reset_index()

C:\Users\Asus\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: FutureWarning:

Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.



In [292]:
loc=dfNew.groupby(['Postcode','Age','Policy Value'])['index'].median()
loc=loc.reset_index()
loc

,Postcode,Age,Policy Value,index
0,04-358,30,813464.0,989
1,05-094,71,490579.0,268
2,05-280,75,1237.0,201
3,05-280,75,49555.0,201
4,05-340,65,903915.0,418
...,...,...,...,...
689,N1G,70,31476.0,284
690,N9A,82,238710.0,104
691,P56,67,555607.0,349
692,R42,69,9768.0,321


In [270]:
nomi = pgeocode.Nominatim('US')

In [293]:
state=[]
for i in range(len(loc)):
    state.append(nomi.query_postal_code(loc['Postcode'][i]).state_name)

In [294]:
loc['State Name']=state

In [295]:
loc_avail=loc[loc['State Name'].notnull()]

In [303]:
loc_avail1=loc_avail.groupby('State Name')['Policy Value','Age'].median()
loc_avail1=loc_avail1.reset_index()
loc_avail1

C:\Users\Asus\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: FutureWarning:

Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.



,State Name,Policy Value,Age
0,Arizona,950135.0,34.0
1,Arkansas,116295.0,64.0
2,California,263916.5,58.5
3,Colorado,644349.0,62.0
4,District of Columbia,742026.0,34.0
5,Florida,66815.0,65.0
6,Georgia,694305.0,71.0
7,Illinois,445292.0,61.0
8,Indiana,368868.5,62.0
9,Iowa,677211.0,53.0


In [304]:
fig = px.bar(loc_avail1, y='Policy Value', x='State Name', text='Policy Value', color='Age')
fig.update_traces(texttemplate='%{text:.2s}', textposition='outside')
fig.update_layout(uniformtext_minsize=8, uniformtext_mode='hide')
fig.show()